# Introduction

In this lab session, you will be improving your practical understanding of neural machine translation (NMT). For this, you will experiment with state-of-the-art recurrent neural machine translation (NMT) models. For training, you will be using the [Multi30k](https://github.com/multi30k/dataset) dataset, which contains around 30,000 image descriptions in English and their translations in German, French and Czech. Although Multi30k is often used for multimodal machine translation, it is a good candidate for quick MT experiments due to its smaller size.

## 1. Downloading the corpus

Let's start by downloading the corpus. We'll also install `sacreBLEU` for BLEU computation.

In [ ]:
%%bash

# install sacreBLEU
pip install sacrebleu==1.5.0
echo

# Download the corpus
URL="https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/tok"

for split in "train" "val" "test_2016_flickr"; do
  for lang in en de fr; do
    fname="${split}.lc.norm.tok.${lang}"
    if [ ! -f $fname ]; then
      echo "Downloading $fname"
      wget -q "${URL}/$fname" -O "${split/_2016_flickr/}.${lang}"
    fi
  done
done
echo 

# Print the first 10 lines with line numbers of 
# the English and French training data
cat -n train.en | head -n10
echo

cat -n train.fr | head -n10
echo


## 2. Setting up the environment

The next step is to set up the environment for `pytorch` and define some helper functions. 

In [ ]:
import math
import time

import numpy as np

import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence, pack_sequence

import sacrebleu

###############
# Torch setup #
###############
print('Torch version: {}, CUDA: {}'.format(torch.__version__, torch.version.cuda))
cuda_available = torch.cuda.is_available()

if not torch.cuda.is_available():
  print('WARNING: You may want to change the runtime to GPU for faster training!')
  DEVICE = 'cpu'
else:
  DEVICE = 'cuda:0'

#######################
# Some helper functions
#######################
def fix_seed(seed=None):
  """Sets the seeds of random number generators."""
  if seed is None:
    # Take a random seed
    seed = time.time()
  seed = int(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  return seed

def readable_size(n):
  """Returns a readable size string for model parameters count."""
  sizes = ['K', 'M', 'G']
  fmt = ''
  size = n
  for i, s in enumerate(sizes):
    nn = n / (1000 ** (i + 1))
    if nn >= 1:
      size = nn
      fmt = sizes[i]
    else:
      break
  return '%.2f%s' % (size, fmt)

## 3. Representing the vocabulary

The `Vocabulary` class, which is defined below, encapsulates the **word-to-idx** and **idx-to-word** mapping that you should now be familiar with from the previous lab sessions.

In [ ]:
class Vocabulary(object):
  """Data structure representing the vocabulary of a corpus."""
  def __init__(self):
    # Mapping from tokens to integers
    self._word2idx = {}

    # Reverse-mapping from integers to tokens
    self.idx2word = []

    # 0-padding token
    self.add_word('<pad>')
    # sentence start
    self.add_word('<s>')
    # sentence end
    self.add_word('</s>')
    # Unknown words
    self.add_word('<unk>')

    self._pad_idx = self._word2idx['<pad>']
    self._bos_idx = self._word2idx['<s>']
    self._eos_idx = self._word2idx['</s>']
    self._unk_idx = self._word2idx['<unk>']

  def word2idx(self, word):
    """Returns the integer ID of the word or <unk> if not found."""
    return self._word2idx.get(word, self._unk_idx)

  def add_word(self, word):
    """Adds the `word` into the vocabulary."""
    if word not in self._word2idx:
      self.idx2word.append(word)
      self._word2idx[word] = len(self.idx2word) - 1

  def build_from_file(self, fname):
    """Builds a vocabulary from a given corpus file."""
    with open(fname) as f:
      for line in f:
        words = line.strip().split()
        for word in words:
          self.add_word(word)

  def convert_idxs_to_words(self, idxs, until_eos=False):
    """Converts a list of indices to words."""
    if until_eos:
      try:
        idxs = idxs[:idxs.index(self.word2idx('</s>'))]
      except ValueError:
        pass

    return ' '.join(self.idx2word[idx] for idx in idxs)

  def convert_words_to_idxs(self, words, add_bos=False, add_eos=False):
    """Converts a list of words to a list of indices."""
    idxs = [self.word2idx(w) for w in words]
    if add_bos:
      idxs.insert(0, self.word2idx('<s>'))
    if add_eos:
      idxs.append(self.word2idx('</s>'))
    return idxs

  def __len__(self):
    """Returns the size of the vocabulary."""
    return len(self.idx2word)

  def __repr__(self):
    return "Vocabulary with {} items".format(self.__len__())

## 4. Representing the corpus

Below we define a `Multi30K` class that encapsulates the vocabularies and the source/target sides of train/dev/test splits. Two important methods are `read_sentences()` and `get_batches()` which you may want to read in detail. 

***Q1: In the `get_batches()` method, explain why `pad_sequence` is used.***

**A:** To construct a proper batch tensor, word indices should be padded to the length of the longest sequence.

In [ ]:
class Multi30K:
  """A dataset wrapper for Multi30K."""
  def __init__(self, src_lang='en', trg_lang='fr'):
    self.src_lang = src_lang
    self.trg_lang = trg_lang

    # Create vocabularies
    self.src_vocab = Vocabulary()
    self.src_vocab.build_from_file(f'train.{src_lang}')
    self.trg_vocab = Vocabulary()
    self.trg_vocab.build_from_file(f'train.{trg_lang}')

    self.n_src_vocab = len(self.src_vocab)
    self.n_trg_vocab = len(self.trg_vocab)

    self._data = {}
    for split in ('train', 'val', 'test'):
      # Read sentences and map to indices using the vocabularies
      print(f'Reading {split} set')
      self._data[split] = self.read_sentences(split)

  def read_sentences(self, split):
    src_sents = []
    trg_sents = []

    # Read source side
    with open(f'{split}.{self.src_lang}') as f:
      for line in f:
        line = line.strip()
        assert line, "Empty line found, please fix this!"
        idxs = self.src_vocab.convert_words_to_idxs(line.split(), add_eos=True)
        src_sents.append(idxs)
    # Read source side
    with open(f'{split}.{self.trg_lang}') as f:
      for line in f:
        line = line.strip()
        assert line, "Empty line found, please fix this!"
        idxs = self.trg_vocab.convert_words_to_idxs(line.split(), add_bos=True, add_eos=True)
        trg_sents.append(idxs)

    assert len(src_sents) == len(trg_sents), "Files are not aligned!"
    return src_sents, trg_sents

  def get_batch(self, idxs, split='train'):
    """Returns padded torch tensors for source and target sample indices."""
    src_idxs = [torch.LongTensor(self._data[split][0][i]) for i in idxs]
    trg_idxs = [torch.LongTensor(self._data[split][1][i]) for i in idxs]

    ###################################
    # Pad sequences to longest sequence
    ###################################
    padded_src_idxs = pad_sequence(src_idxs, padding_value=self.src_vocab._pad_idx)
    padded_trg_idxs = pad_sequence(trg_idxs, padding_value=self.trg_vocab._pad_idx)
    return padded_src_idxs.to(DEVICE), padded_trg_idxs.to(DEVICE)

  def __repr__(self):
    s = f"Multi30K {self.src_lang} (# {self.n_src_vocab}) -> {self.trg_lang} (# {self.n_trg_vocab})\n"
    s += f" train: {len(self._data['train'][0])} sentences\n"
    s += f"   val: {len(self._data['val'][0])} sentences\n"
    s += f"  test: {len(self._data['test'][0])} sentences"
    return s

## 5. Create the dataset

We now create a dataset instance to perform translation from French (FR) to English (EN). For demonstration purposes, we request a toy batch with 5 sentences and convert indices back to word forms.

Notice how each column represents sentences and the shorter sequences are **zero-padded** at right.

**NOTE**: Any code that deal with sequence-to-sequence paradigms should do its best to ignore these positions correctly, especially when using RNNs and Transformers with attention layers. Some support may come from the underlying toolkit that is used, through arguments related to **padding masks**.

In [ ]:
dataset = Multi30K(src_lang='fr', trg_lang='en')
print(dataset)

# Get a batch and see inside
src_idxs, trg_idxs = dataset.get_batch([0, 10, 234, 12, 7], split='val')
print(src_idxs)
for i in range(src_idxs.shape[1]):
  print('SRC: ', dataset.src_vocab.convert_idxs_to_words(src_idxs[:, i]))
  print('TRG: ', dataset.trg_vocab.convert_idxs_to_words(trg_idxs[:, i]))
  print()

# 6. Encoder-Decoder Model

In this section, you will implement an encoder-decoder model for neural machine translation (NMT). For a given source sentence $X = \{x_1, \dots, x_S\}$ and the corresponding translation $Y = \{y_1, \dots, y_T\}$, the model tries to predict the probability of words in the translation $Y$ using a **vectorial representation** $v$ of the source sentence.

- The encoder is straight-forward: it receives the embeddings of the words in $X$ and produces contextualised representations for each position in the form of a matrix $H = \{h_1, \dots, h_S\}$. The encoder can be uni- or bi-directional.

- Several approaches exist to compress the whole source sentence into a single vector $v$:
  - **Max pooling:** Perform a max pooling over $H$ to select the highest activations through time for each dimension.
  - **Average pooling:** Take the average state as the representation by setting $v = \frac{1}{S}\sum_{i=1}^S h_i$.
  - **Last hidden state:** Set $v = h_S$ assuming that the last hidden state of the RNN encoder would have collected as much linguistic information as possible from the source sentence.

- The decoder is exactly a neural LM which is **additionally conditioned** over the representation vector $v$. The simplest way of achieving this is to set the  initial hidden state $d_0$ of the decoder to the (non-) linear projection of $v$.

---

**Q2: Follow the `EncDecNMT` class below and fill in the `<TODO>` items:**

- **Line ~49:** Create the GRU encoder
- **Line ~59:** Handle encoders' output dimension in bi-directional case

In [ ]:
class EncDecNMT(nn.Module):
  """Encoder-decoder NMT without attention."""
  def __init__(self, dataset, emb_dim, enc_dim, dec_dim,
               enc_bidirectional=False,
               init_dec='max',
               dropout=0.3, clip_gradient_norm=1.0, tie_weights=True,
               batch_size=64):
    # Call parent's __init__ first
    super(EncDecNMT, self).__init__()

    # Store arguments
    self.dataset = dataset
    self.emb_dim = emb_dim
    self.enc_dim = enc_dim
    self.enc_bidirectional = enc_bidirectional
    self.dec_dim = dec_dim
    self.init_dec = init_dec
    self.clip_gradient_norm = clip_gradient_norm
    self.p_dropout = dropout
    self.tie_weights = tie_weights
    self.batch_size = batch_size

    assert self.init_dec in ('max', 'avg'), \
      "init_dec argument contains unknown value!"

    # Since target sequences are <pad>'ded, we want to ignore the loss
    # values for those positions
    self.loss = nn.CrossEntropyLoss(
        reduction='none', ignore_index=self.dataset.trg_vocab._pad_idx)

    # Create the dropout
    self.drop = nn.Dropout(p=self.p_dropout)

    ###############################################
    # Create the source and target embedding layers
    ###############################################
    self.src_emb = nn.Embedding(
      num_embeddings=self.dataset.n_src_vocab, embedding_dim=self.emb_dim,
      padding_idx=self.dataset.src_vocab._pad_idx)
    self.trg_emb = nn.Embedding(
      num_embeddings=self.dataset.n_trg_vocab, embedding_dim=self.emb_dim,
      padding_idx=self.dataset.trg_vocab._pad_idx)

    ###########################################
    # QUESTION
    ###########################################
    # Create the encoder by using the arguments
    ###########################################
    #self.enc = nn.GRU("<TODO>")
    self.enc = nn.GRU(
        input_size=self.emb_dim, hidden_size=self.enc_dim,
        num_layers=1, bidirectional=self.enc_bidirectional,
    )

    ##################################################################
    # QUESTION
    ##################################################################
    # Compute encoder's output dim which may be different than enc_dim
    # because of bidirectionality
    ##################################################################
    self.enc_out_dim = self.enc_dim
    if self.enc_bidirectional:
      #self.enc_out_dim = "<TODO>"
      self.enc_out_dim *= 2

    ###############################################################
    # Let's use the `GRUCell` for decoder, which is designed to run for
    # single timesteps through an explicit for loop. This will make
    # it easier to understand the concepts shown at the lecture.
    ###############################################################
    self.dec = nn.GRUCell(input_size=self.emb_dim, hidden_size=self.dec_dim)

    ##############################################################
    # Add a non-linear layer for decoder initialisation projection
    ##############################################################
    self.ff_dec_init = nn.Sequential(
        nn.Linear(self.enc_out_dim, self.dec_dim),
        nn.Tanh(),
    )

    ####################################################################
    # Bottleneck layer maps decoder's hidden state to emb size for tying
    ####################################################################
    self.bneck = nn.Linear(self.dec_dim, self.emb_dim)

    ####################
    # Final output layer
    ####################
    self.out = nn.Linear(self.emb_dim, self.dataset.n_trg_vocab)

    ########################################################################
    # This cuts the number of parameters by sharing the same embedding layer
    # for both the inputs and outputs to/from the decoder GRU
    ########################################################################
    if self.tie_weights:
      self.out.weight = self.trg_emb.weight

    # Reset padding embeddings to all 0s for sanity
    with torch.no_grad():
        self.src_emb.weight.data[0].fill_(0)
        self.trg_emb.weight.data[0].fill_(0)

  def __repr__(self):
    """String representation for pretty-printing."""
    n_params = 0
    for param in self.parameters():
      n_params += np.cumprod(param.data.size())[-1]
    n_params = readable_size(n_params)

    s = super(EncDecNMT, self).__repr__()
    return f"{s}\n# of parameters: {n_params} -- Decoder init: '{self.init_dec}'"

  def get_batch_indices(self, split='train', shuffle=True):
    """Returns the list of sample indices for a whole epoch."""
    # Get number of samples
    n_samples = len(self.dataset._data[split][0])

    # Get sample indices and batch them
    if shuffle:
      order = torch.randperm(n_samples)
    else:
      order = torch.arange(n_samples)

    start_offsets = range(0, n_samples, self.batch_size)
    return [order[i: i + self.batch_size] for i in start_offsets]

  ###################################
  # Decoder initialisation approaches
  ###################################
  def get_encoder_max_state(self, enc_states, input_mask):
    """Computes h_0 for the decoder based on the max-pooled encoding state."""
    # we fill the padded position values with small numbers so that
    # max-pooling is not able to return wrong max values
    masked_states = enc_states.masked_fill(
        input_mask.unsqueeze(-1).bool().logical_not(), -100000)
    max_state = masked_states.max(0)[0]
    return max_state

  def get_encoder_avg_state(self, enc_states, input_mask):
    """Computes the average of encoder states taking care of padding."""
    ##############################################################
    # QUESTION
    ##############################################################
    # Compute the average encoder states by taking into account
    # sentence length information through `input_mask`
    ##############################################################
    # raise RuntimeError('Not implemented yet!')
    return enc_states.sum(0).div(input_mask.sum(0).unsqueeze(-1))

  def compute_decoder_state(self, enc_states, input_mask):
    """Calls the appropriate `init_dec` method, projects the `v`."""
    func = getattr(self, f'get_encoder_{self.init_dec}_state')
    # Get the vector `v`
    h_0 = func(enc_states, input_mask)
    # Project it with the FF
    return self.ff_dec_init(h_0)

  ####################################
  # Encodes a batch of input sentences
  ####################################
  def encode(self, x):
    """Encode tokens `x` to obtain the encoder states."""
    # Compute the mask to detect <pad>'s in the further parts
    # x is a padded tensor of shape (seq_len, batch_size)
    # mask: (seq_len, batch_size)
    self.mask = x.ne(self.dataset.src_vocab._pad_idx).long()

    # src_embs: (seq_len, batch_size, emb_dim)
    embs = self.drop(self.src_emb(x))

    # Pack the tensor so that RNN correctly computes the hidden
    # states by ignoring padded positions
    packed_inputs = pack_padded_sequence(
        embs, lengths=self.mask.sum(0).long().cpu(), enforce_sorted=False)

    # Encode -> unpack to obtain an ordinary tensor of hidden states
    # padded positions will now have explicit 0's in their hidden states
    # all_hids: (seq_len, batch_size, self.enc_out_dim)
    self.all_hids = pad_packed_sequence(self.enc(packed_inputs)[0])[0]
    return self.all_hids, self.mask

  def compute_loss_from_logits(self, logits, y):
    """Returns the scalar losses for every element in the batch."""
    return self.loss(logits, y)

  def compute_decoder_logits(self, dec_hid_state, y):
    # *Cell() functions return (batch_size, hidden_size) i.e.
    # a tensor containing the computed hidden state for each element in the batch
    dec_hid_state = self.dec(self.trg_emb(y), dec_hid_state)
    logits = self.out(self.bneck(self.drop(dec_hid_state)))
    return logits, dec_hid_state

  def forward(self, x, y):
    """Forward-pass of the model for training/ppl evaluation only."""
    # Encode source sentence and get encoder states
    enc_states, input_mask = self.encode(x)

    # Compute decoder's initial state h_0 for each sentence
    dec_hid_state = self.compute_decoder_state(enc_states, input_mask)

    # Manually compute the loss for each decoding timestep
    # We skip the last item since we don't want to input </s>
    loss = 0.0
    n_trg_tokens = 0
    for t in range(y.size(0) - 1):
      y_prev, y_next = y[t], y[t + 1]

      # Do a recurrence step
      logits, dec_hid_state = self.compute_decoder_logits(dec_hid_state, y_prev)

      # Accumulate the sequence loss
      loss += self.compute_loss_from_logits(logits, y_next).sum()

      # Compute number of valid positions by ignoring <pad>'s
      n_trg_tokens += y_next.ne(self.dataset.trg_vocab._pad_idx).sum()

    return loss, float(n_trg_tokens.item())

  def train_model(self, optim, n_epochs=5):
    """Trains the model."""
    train_ppls, val_ppls = [], []

    for eidx in range(1, n_epochs + 1):
      start_time = time.time()
      epoch_loss = 0
      epoch_items = 0

      # Enable training mode
      self.train()

      # Start training (will shuffle at each epoch)
      for iter_count, idxs in enumerate(self.get_batch_indices('train')):
        # Get x's and y's
        x, y = self.dataset.get_batch(idxs)

        # Clear the gradients
        optim.zero_grad()

        total_loss, n_items = self.forward(x, y)

        # Backprop the average loss and update parameters
        total_loss.div(n_items).backward()

        # Clip the gradients to avoid exploding gradients
        if self.clip_gradient_norm > 0:
          torch.nn.utils.clip_grad_norm_(self.parameters(), self.clip_gradient_norm)

        # Update parameters
        optim.step()

        # sum the loss for reporting, along with the denominator
        epoch_loss += total_loss.item()
        epoch_items += n_items

        # Overall epoch loss and ppl
        loss_per_token = epoch_loss / epoch_items
        ppl = math.exp(loss_per_token)

        if (iter_count + 1) % 100 == 0:
          # Print progress
          print(f'[Epoch {eidx:<3}] loss: {loss_per_token:6.2f}, perplexity: {ppl:6.2f}')

      time_spent = time.time() - start_time

      print(f'[Epoch {eidx:<3}] ended with train_loss: {loss_per_token:6.2f}, train_ppl: {ppl:6.2f}')
      train_ppls.append(ppl)

      # Evaluate on valid set
      valid_loss, valid_ppl = self.evaluate('val')
      val_ppls.append(valid_ppl)
      print(f'[Epoch {eidx:<3}] ended with valid_loss: {valid_loss:6.2f}, valid_ppl: {valid_ppl:6.2f}')
      print(f'[Epoch {eidx:<3}] completed in {time_spent:.2f} seconds')
      print(f'[Epoch {eidx:<3}] validation BLEU with greedy search:')

      bleu = self.greedy_search('val')
      print(f'[Epoch {eidx:<3}] {bleu}\n')

    ######################################
    # Evaluate the final model on test set
    ######################################
    test_loss, test_ppl = self.evaluate('test')
    print(f' ---> Final test set performance: {test_loss:6.2f}, test_ppl: {test_ppl:6.2f}')

  def evaluate(self, split):
    # Switch to eval mode
    self.eval()

    eval_loss = 0.
    eval_tokens = 0

    with torch.no_grad():
      for iter_count, idxs in enumerate(self.get_batch_indices(split, shuffle=False)):
        # Get x's and y's
        x, y = self.dataset.get_batch(idxs, split=split)

        total_loss, n_items = self.forward(x, y)
        eval_loss += total_loss.item()
        eval_tokens += n_items
    eval_loss /= eval_tokens

    return eval_loss, math.exp(eval_loss)

  def greedy_search(self, split, max_len=60):
    """Performs a greedy search, dumps the translations and computes BLEU."""
    # Switch to eval mode
    self.eval()

    bos = self.dataset.trg_vocab._bos_idx
    eos = self.dataset.trg_vocab._eos_idx

    # We keep the hypotheses for a batch in a tensor for efficiency
    # Although there's a hard-limit for decoding timesteps `max_len`,
    # the hypotheses will likely to produce </s> before reaching `max_len`.
    batch_hyps = torch.zeros(
      (max_len, self.batch_size), dtype=torch.long,
      device=self.trg_emb.weight.device)

    # Resulting sentences in dataset split order
    results = []

    with torch.no_grad():
      for iter_count, idxs in enumerate(self.get_batch_indices(split, shuffle=False)):
        # We don't care about `y` for translation decoding
        # i.e. we rely on model's own predictions rather than ground-truths
        x, _ = self.dataset.get_batch(idxs, split=split)

        # Clear batch hypotheses tensor
        batch_hyps.zero_()

        # Get encoder hidden states
        enc_states, input_mask = self.encode(x)

        # Compute decoder's initial state h_0 for each sentence
        h = self.compute_decoder_state(enc_states, input_mask)

        # last batch could be smaller than the requested batch size
        cur_batch_size = h.size(0)

        # Start all sentences with <s>
        next_word_idxs = torch.full(
            (cur_batch_size, ), bos, dtype=torch.long, device=h.device)

        # Track sentences who already produced </s>
        track_fini = torch.zeros((cur_batch_size, ), device=h.device).bool()

        # A maximum of `max_len` decoding steps
        for t in range(max_len):
          if track_fini.all():
            # All hypotheses produced </s>, early stop!
            break

          # Get logits from the decoder
          logits, h = self.compute_decoder_logits(h, next_word_idxs)

          # Get next probabilities and argmax them for every sentence
          next_word_idxs = nn.functional.softmax(logits, dim=-1).argmax(dim=-1)

          # Update finished sentence tracker
          track_fini.add_(next_word_idxs.eq(eos))

          # Insert most probable words for timestep `t` into tensor
          batch_hyps[t, :cur_batch_size] = next_word_idxs

        # All finished, convert translations to python lists on CPU
        results.extend(batch_hyps[:, :cur_batch_size].t().cpu().tolist())

    # post-process results to convert them to actual sentences
    out_fname = f'{split}_translations.{self.dataset.trg_lang}'
    ref_fname = f'{split}.{self.dataset.trg_lang}'
    hyps = []
    refs = []

    # read references
    with open(ref_fname) as f:
      for line in f:
        refs.append(line.strip())

    # Write hyps
    with open(out_fname, 'w') as f:
      for sent in results:
        sent_str = self.dataset.trg_vocab.convert_idxs_to_words(sent, True)
        hyps.append(sent_str)
        f.write(sent_str + '\n')

    # Compute BLEU
    bleu = sacrebleu.corpus_bleu(hyps, [refs], tokenize='none')
    return bleu

## 7. Training the model

The following function trains an encoder-decoder NMT with the base hyper-parameters that you can override through the keyword arguments. Proceed to the next code block for the actual training.

In [ ]:
def train_encdec_model(n_epochs=5, init_lr=0.0005, **kwargs):
  # Set the seed for reproducible results
  fix_seed(30494)

  base_params = {
    'dataset':dataset,
    'emb_dim':200,              # word embedding dim
    'enc_dim':200,              # hidden layer dim for the encoder
    'enc_bidirectional':False,  # True makes the encoder bidirectional
    'dec_dim':300,              # hidden layer dim for the decoder
    'clip_gradient_norm':1.0,   # gradient clip threshold
    'dropout':0.3,              # dropout probability
    'tie_weights':True,         # Weight typing for decoder inputs/outputs
    'batch_size':64,            # Batch size
    'init_dec':'max',          # Initialize the decoder with max or avg state
  }

  # Override with given arguments
  base_params.update(kwargs)

  # Create the Pytorch model
  model = EncDecNMT(**base_params)

  # move to device
  model.to(DEVICE)

  # Create the optimizer
  opt = torch.optim.Adam(model.parameters(), lr=init_lr)
  print(model)

  # Returns train, val and test perplexities
  model.train_model(opt, n_epochs=n_epochs)
  return model

Below, you can add different configurations to `param_set` and they will be trained sequentially. At the end, the final model parameters will be used to decode the translations for the test set and BLEU scores will be computed and printed for each configuration.

In [ ]:
# add more configurations to here to train other systems
param_set = [
  {'enc_bidirectional': False, 'init_dec': 'max'},
]

bleu_scores = []

for params in param_set:
  header_msg = f'Training {params}'
  print('*' * len(header_msg) + '\n' + header_msg + '\n' + '*' * len(header_msg))

  # Train for 5 epochs
  model = train_encdec_model(n_epochs=5, init_lr=0.0005, **params)

  # Translate the test set and get the bleu scores
  bleu = model.greedy_search('test', max_len=60)
  bleu_scores.append(bleu)
  
print()

for config, score in zip(param_set, bleu_scores):
  print(f'{config}\n  {score}')

## Exercises


At the end of 5 epochs, the default settings with max-pooled decoder initialisation and uni-directional encoder, should give you a BLEU score of $\sim$**19.7** on the test set. Now here are some exercises that you can proceed with:

---

**Q3: Implement the `avg` initialisation (`get_encoder_average_state()` method) and fill in the table below for uni-directional and bi-directional encoder by training more configurations. What are your conclusions? In the end, which initialisation scheme yields the best BLEU?**

**A:** *The results are given in the below table. Note that, we can not conclude by training any model with just 5 epochs and a fixed random seed. In the ideal case, we would have to run it until early-stopping and with several random seeds and compute average and standard deviation of BLEU scores. But in this particular exercise, we notice that bi-directionality seems to add a solid 2-3 BLEU score for both types of initialisation.*

| $v$| Unidir.   | Bidir.   |
|----|-----------|----------|
| max| 19.70      | 22.41     |
|avg | 20.30      | 22.44     |

# 8. Incorporating `DOT` attention

In the following, we derive our attentive NMT model from the `EncDecNMT` and only modify the necessary parts to incorporate the attention mechanism. Specifically, three `Linear` layers are added for decoder's state (query) projection, encoder state (key) projection, and a final projection from encoder's states to output layer dimensionality.

---
**Q4: Follow the code below and fill two `<TODO>` items in `__init__()` and five more in `compute_decoder_logits()` for the dot attention. (The MLP attention parts are extra exercises that you can come back to, after finishing the other questions.)**

In [ ]:
class AttentionNMT(EncDecNMT):
  """Encoder-decoder NMT with attention."""
  def __init__(self, **kwargs):
    # The internal dimension for the dot product i.e.
    # the common dimension that d_t and h_i's should be projected
    self.att_dim = kwargs.pop('att_dim')

    # The attention type to compute the similarity scores
    self.att_type = kwargs.pop('att_type')
    assert self.att_type in ('dot', 'mlp'), "att_type unknown."

    # Call parent's __init__ with the remaining arguments
    super(AttentionNMT, self).__init__(**kwargs)

    ############################################
    # QUESTION
    ############################################
    # Add decoder state (query) projection layer
    #self.ff_q_proj = nn.Linear("<TODO>")
    self.ff_q_proj = nn.Linear(self.dec_dim, self.att_dim)

    ############################################
    # QUESTION
    ############################################
    # Add encoder states projection layer for similarity computation
    ############################################
    #self.ff_k_proj = nn.Linear("<TODO>")
    self.ff_k_proj = nn.Linear(self.enc_out_dim, self.att_dim)

    ####################################################
    # Adaptor so that the output of attention can be fed
    # directly to the `self.bneck` layer
    ####################################################
    self.ff_enc2bneck = nn.Linear(self.enc_out_dim, self.dec_dim)

    if self.att_type == 'mlp':
      ###############################################################
      # EXTRA QUESTION
      # the only parameter you would add is linear layer (no bias)
      # representing the `a` vector in the lecture slides.
      ###############################################################
      #raise RuntimeError('Not implemented yet!')
      self.mlp_att = nn.Linear(self.att_dim, 1, bias=False)

  def encode(self, x):
    # Let's first call the EncDec's encode()
    all_hids, mask = super(AttentionNMT, self).encode(x)

    # This is to avoid projection of encoder states at each decoding step
    # since they can be precomputed at once
    self.e_proj = self.ff_k_proj(all_hids)

    return all_hids, mask

  def compute_decoder_logits(self, dec_hid_state, y):
    ###########################################################
    # This step is the same as encoder-decoder, we feed the embedding
    # and get `d_t` (query for attention) i.e. the hidden state of the decoder
    ###########################################################
    dec_hid_state = self.dec(self.trg_emb(y), dec_hid_state)

    # Below you'll have to do a lot of permute(), t(), squeeze(), unsqueeze()
    # operations to make dimensions compatible. Check PyTorch documents
    # if you are not familiar with these operations

    ###########################################################
    # QUESTION
    ###########################################################
    # Project `dec_hid_state to attention dim with `ff_q_proj` layer
    # Expected shape: (batch_size, att_dim, 1)
    ###########################################################
    #proj_q = "<TODO>"
    proj_q = self.ff_q_proj(dec_hid_state).unsqueeze(-1)

    ###########################################################
    # QUESTION
    ###########################################################
    # Permute the dimensions of already cached `self.e_proj`
    # so that the shape becomes: (batch_size, seq_len, att_dim)
    ###########################################################
    #proj_e = "<TODO>"
    proj_e = self.e_proj.permute(1, 0, 2)

    ###########################################################
    # QUESTION (Dot attention)
    ###########################################################
    # Now that you have the queries for all the batch (proj_q)
    # and encoder states for all source positions in the batch (proj_e)
    # you can use `torch.bmm()` to compute all similarity scores at once.
    # `bmm` stands for "Batch matrix multiplication". If you have two 3D
    # tensors where first dimension represents the `batch_size`, `bmm`
    # computes the products for each element in the batch.
    ##########
    # Example:
    ##########
    # torch.bmm("tensor of size B x S x A" , "tensor of size B x A x 1")
    #    --> produces a tensor of "B x S x 1"
    ##########
    # Use this to obtain the similarity scores and use squeeze() and t()
    # to make it look like (seq_len, batch_size)
    #scores = "<TODO>"
    if self.att_type == 'dot':
      scores = torch.bmm(proj_e, proj_q).squeeze(-1).t()
    elif self.att_type == 'mlp':
      ###########################################################
      # EXTRA QUESTION
      ###########################################################
      #raise RuntimeError('Not implemented yet!')
      x = torch.tanh(proj_e + proj_q.permute(0, 2, 1))
      scores = self.mlp_att(x).squeeze(-1).t()

    #########################################################
    # we fill the padded positions with small numbers so that
    # softmax() does not assign probabilities to them.
    #########################################################
    scores.masked_fill_(self.mask.logical_not(), -1e8)

    #############################################
    # QUESTION
    #############################################
    # Use softmax() on `scores` to obtain alpha's / probabilities
    # expected shape: (seq_len, batch_size)
    #alpha = "<TODO>"
    alpha = scores.softmax(0)

    #############################################
    # QUESTION
    #############################################
    # Weigh the bidirectional encoder states `self.all_hids`
    # with `alpha`
    # expected shape: (batch_size, self.enc_out_dim)
    #ctx = "<TODO>"
    ctx = (alpha.unsqueeze(-1) * self.all_hids).sum(0)

    # Project the computed weighted context to `dec_dim` so that
    # the output layer works as espected
    # shape: (batch_size, dec_dim)
    c_t = self.ff_enc2bneck(ctx)

    ##################################################################
    # We sum the decoder's state `d_t` and the computed `c_t` together
    ##################################################################
    logits = self.out(self.bneck(self.drop(c_t + dec_hid_state)))
    return logits, dec_hid_state

## 9. Training the model

The following function trains an attentive NMT with the base hyper-parameters that you can override through the keyword arguments. Proceed to the next code block for the actual training.

In [ ]:
def train_attention_model(n_epochs=5, init_lr=0.0005, **kwargs):
  # Set the seed for reproducible results
  fix_seed(30494)

  base_params = {
    'dataset':dataset,
    'emb_dim':200,              # word embedding dim
    'enc_dim':200,              # hidden layer dim for the encoder
    'enc_bidirectional':True,   # True makes the encoder bidirectional
    'dec_dim':300,              # hidden layer dim for the decoder
    'clip_gradient_norm':1.0,   # gradient clip threshold
    'dropout':0.3,              # dropout probability
    'tie_weights':True,         # Weight typing for decoder inputs/outputs
    'batch_size':64,            # Batch size
    'init_dec':'avg',           # Initialize the decoder with max or avg state
    'att_dim': 200,             # Dot product's inner dimension
    'att_type': 'dot',          # att_type dot/mlp
  }

  # Override with given arguments
  base_params.update(kwargs)

  # Create the Pytorch model
  model = AttentionNMT(**base_params)

  # move to device
  model.to(DEVICE)

  # Create the optimizer
  opt = torch.optim.Adam(model.parameters(), lr=init_lr)
  print(model)

  # Returns train, val and test perplexities
  model.train_model(opt, n_epochs=n_epochs)
  return model

Below, you can add different configurations to `param_set` and they will be trained sequentially. At the end, the final model parameters will be used to decode the translations for the test set and BLEU scores will be computed and printed for each configuration.

In [ ]:
# add more configurations to here to train other systems
param_set = [
  {'enc_bidirectional': True, 'init_dec': 'avg', 'att_dim': 200, 'att_type': 'dot'},
  {'enc_bidirectional': True, 'init_dec': 'avg', 'att_dim': 200, 'att_type': 'mlp'},
]

bleu_scores = []

for params in param_set:
  header_msg = f'Training {params}'
  print('*' * len(header_msg) + '\n' + header_msg + '\n' + '*' * len(header_msg))

  # Train for 5 epochs
  model = train_attention_model(n_epochs=5, init_lr=0.0005, **params)

  # Translate the test set and get the bleu scores
  bleu = model.greedy_search('test', max_len=60)
  bleu_scores.append(bleu)

print()

for config, score in zip(param_set, bleu_scores):
  print(f'{config}\n  {score}')

# Conclusion

After completing the model and training the provided configuration, you should see a test BLEU of around **41.4** after 5 epochs! It's almost twice the performance that we observed with the decoder initialisation method.

You can now play with the hyper-parameters to discover a better model in terms of the BLEU score or proceed with the extra questions below!


# Extra Questions

**Q5:** Modify `EncDecNMT` to add a new initalisation method called `each`. In this mode, instead of initialising the decoder's initial state with the average encoder state, you'll add the average encoder state to each `y_prev` i.e. to each input of the decoder. This way, the same information will be much more accessible throughout decoding, instead of just being used at $t=0$. How does the performance compare to the `avg` method? Do you think initialising the decoder's hidden state is crucial to pass along source sentence representation or are you satisfied with this approach as well?

**NOTE:** Make sure that you do no longer initialise your decoder's hidden state with the same information and leave it `None` i.e. a 0-vector initialisation. You can do this by adding a new initialisation method `zero` to your class and implementing the rest accordingly.

**ANS:** *You should get approximately the same BLEU score ($22.8$ instead of $22.4$) when using this new method. This means that there can be many other ways of passing source side information to the decoder other than the hidden state initialisation.*

---

**Q6: (Attentive NMT)** Experiment and try to come up with a better hyper-parameters settings that surpasses the default setting in terms of test BLEU. For example, you can try increasing the number of layers in the encoder to see what happens.

**ANS:** *You should get a test BLEU of **43.02** with a stacked 2-layer GRU encoder, instead of 41.4*.

---

**Q7: (Attentive NMT)** Fill in the missing parts to implement the MLP attention. Compare the performance to `dot` attention by keeping all other hyper-parameters the same.

**ANS:** Although MLP attention gives a slightly lower BLEU score 40.6 after 5 epochs, no conclusions should be drawn unless models are trained until completion or early-stopping.

---

**Q8: (Attentive NMT)** Try implementing the idea called [input feeding](https://arxiv.org/abs/1508.04025). In this model, you will project the final context vector $c_t$ to $c'_t$ (to match decoder's input embedding dimensionality) and add it to the embeddings $y_t$ in the next step. This way, the computed context will recur through time as well. Specifically, the input to the decoder at each timestep will now be $(y_t + c'_t)$ with $c'_0 = 0$ for the very first timestep of decoding.

**ANS:** *If everything goes well, you should obtain approximately **42.6** BLEU with `dot` attention. The modifications required to implement input feeding are as follows:*

```python
# (1) Add another linear layer into __init__
self.ff_feed_proj = nn.Linear(self.dec_dim, self.emb_dim)

# (2) Store a zero vector in self.encode, to represent c'_0 
self.i_feed = all_hids.new_zeros(all_hids.size(1), self.emb_dim)

# (3) sum the inputs to decoder at the beginning of compute_decoder_logits()
dec_hid_state = self.dec(self.trg_emb(y) + self.i_feed, dec_hid_state)

# (4) In compute_decoder_logits(), compute c'_t right after c_t
self.i_feed = self.ff_feed_proj(c_t)
```

---

**Q9: (Attentive NMT)** Do you think initialising the decoder's hidden state is still necessary & helpful for an attentive NMT? Try the following: add a new initialisation method `zero` and change the code accordingly so that the `GRUCell`'s forward call receives `None` as the hidden state argument. A `None` means that internally, a zero-vector will be used to initialise the decoder's hidden state. Train the `dot` attention with `zero` initialisation method and compare the results.

**ANS:** Although we don't provide numbers here, the model should still be doing fine without showing substantial drops in MT performance. So once the attention is there, initialisation of decoder's state becomes less important.
